In [1]:
from modelUnetVGG19 import *
from data_3ch import *

vid="v061_5" #version id should match the file number 
# The final number can be [1:5] and denotes the cross-validation order
# Ran DellWS with GeForce RTX3060 GPU

### Train with data generator

In [2]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger

#Data augmentation
data_gen_args = dict(rotation_range=45,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    vertical_flip=True,
                    fill_mode='wrap')

batch_size=3
LR=1e-4
loss=dice_loss

train_gen = trainGenerator(batch_size,'afm_cell_512/train0'+vid[-1],'afmheight','labels',data_gen_args,save_to_dir = None)
valid_gen = trainGenerator(batch_size,'afm_cell_512/valid0'+vid[-1],'afmheight','labels',data_gen_args,save_to_dir = None)

# 24 images are used for training, 6 images for validating and 4 images for testing
train_steps = 24//batch_size
valid_steps = 6//batch_size

model = build_vgg19_unet()

opt = tf.keras.optimizers.Adam(LR)
metrics = ["acc", dice_coef, iou]
model.compile(loss=loss, optimizer=opt, metrics=metrics)

callbacks = [
             ModelCheckpoint('files/unet_cell_seg_'+vid+'.hdf5', verbose=1, save_best_model=True),
             ReduceLROnPlateau(monitor="val_loss", patience=3, factor=0.1, verbose=1, min_lr=1e-8),
             CSVLogger("files/data_"+vid+".csv"),
             EarlyStopping(monitor="val_loss", patience=5, verbose=1)
            ]

model.fit_generator(train_gen, validation_data=valid_gen, steps_per_epoch=10*train_steps, validation_steps=valid_steps, 
                    epochs=100, callbacks=callbacks)


C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Found 24 images belonging to 1 classes.
Found 24 images belonging to 1 classes.
Epoch 1/100
80/80 [==============================] - ETA: 0s - loss: 0.2368 - acc: 0.9212 - dice_coef: 0.7632 - iou: 0.6414Found 6 images belonging to 1 classes.
Found 6 images belonging to 1 classes.
80/80 [==============================] - 62s 508ms/step - loss: 0.2368 - acc: 0.9212 - dice_coef: 0.7632 - iou: 0.6414 - val_loss: 0.4742 - val_acc: 0.3518 - val_dice_coef: 0.5258 - val_iou: 0.3594

Epoch 00001: saving model to files\unet_cell_seg_v061_5.hdf5
Epoch 2/100
80/80 [==============================] - 39s 493ms/step - loss: 0.1282 - acc: 0.9627 - dice_coef: 0.8718 - iou: 0.7803 - val_loss: 0.3759 - val_acc: 0.9039 - val_dice_coef: 0.6241 - val_iou: 0.4566

Epoch 00002: saving model to files\unet_cell_seg_v061_5.hdf5
Epoch 3/100
80/80 [==============================] - 40s 498ms/step - loss: 0.1123 - acc: 0.9668 - dice_coef: 0.8877 - iou: 0.8041 - val_loss: 0.4488 - val_acc: 0.8630 - val_dice_coef: 0.

### validate your model and save predicted results

In [3]:
validGene = testGenerator("afm_cell_512/valid0"+vid[-1]+"/predh")
model.load_weights("files/unet_cell_seg_"+vid+".hdf5")
results = model.predict_generator(validGene,6,verbose=1)
saveResult("afm_cell_512/valid0"+vid[-1]+"/predh",results,vid[:4])

C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:2001: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


6/6 [==============================] - 5s 213ms/step


In [4]:
model.summary()

Model: "VGG19_U-Net"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 512, 512, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 512, 512, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 256, 256, 64) 0           block1_conv2[0][0]               
________________________________________________________________________________________